In [31]:
%load_ext autoreload
%autoreload 2
import sys
import os
import pandas as pd
import datetime as datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import OneHotEncoder
import matplotlib
import matplotlib.pyplot as plt
#import psycopg2
from scipy.stats import ks_2samp
import scipy.stats as scats
# import visuals as vs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

import yaml

from icu_mortality_prediction import DATA_DIR


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
from icu_mortality_prediction.src.features import ptnt_demog
from icu_mortality_prediction.src.utils import data_utils
from icu_mortality_prediction.src.utils import feature_selection

# MIMIC-III Critical Care Database

MIMIC-III (Medical Information Mart for Intensive Care III) is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012.

The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (both in and out of hospital).

MIMIC supports a diverse range of analytic studies spanning epidemiology, clinical decision-rule improvement, and electronic tool development. It is notable for three factors:

it is freely available to researchers worldwide
it encompasses a diverse and very large population of ICU patients
it contains high temporal resolution data including lab results, electronic documentation, and bedside monitor trends and waveforms.

Citations: 
MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available at: http://www.nature.com/articles/sdata201635

Pollard, T. J. & Johnson, A. E. W. The MIMIC-III Clinical Database http://dx.doi.org/10.13026/C2XW26 (2016).



# IMPORTING DATA
The mimic III database was downloaded and reconstructed locally using postgresql. The database was managed graphically using Portico. 
A query was run on the mimic III database to generate demographic data and data concerning hospital and ICU stays for patients diagnosed with sepsis according to the Angus criteria (Angus et al, 2001. Epidemiology of severe sepsis in the United States; http://www.ncbi.nlm.nih.gov/pubmed/11445675 )

The query was exported from Porticoto the file PTNT_DEMOG_FIRST24.csv. The data was read into a pandas dataframe lab_events.

## IMPORTING DATA

In [33]:
# patient demographic data includes diagnoses and icd9 codes for each patient and each icustay
demographics_filename = 'PTNT_DEMOG_FIRST24.csv'
demographics_filepath = os.path.join(DATA_DIR, 'interim',demographics_filename)
ptnt_demog_data = pd.read_csv(demographics_filepath, index_col = 'icustay_id')
ptnt_demog_data = ptnt_demog_data.head(10000)
ptnt_demog_data2 = ptnt_demog_data[~ptnt_demog_data.index.duplicated(keep='first')].copy()
# ptnt_demog_data2.drop(['subject_id', 'deathtime', 'hadm_id'], inplace = True, axis = 1)
# cols = list(ptnt_demog_data2.columns)
# cols.insert(0, cols.pop(cols.index('hospital_expire_flag')))
# ptnt_demog_data2 = ptnt_demog_data2[cols]
print("imported")


imported


In [64]:
feat_df = pd.read_csv(os.path.join(DATA_DIR, 'interim/features/', 'Ptnt_Demog_Features.csv'))

In [65]:
feat_df.head()

,icustay_id,hospital_expire_flag,Respiratory failure; insufficiency; arrest (adult),Shock,Septicemia (except in labor),Acute and unspecified renal failure,icu_stay_Q3,Other liver diseases,first_careunit_CSRU,Fluid and electrolyte disorders,...,ethnicity_UNABLE TO OBTAIN,Chronic obstructive pulmonary disease and bronchiectasis,Chronic kidney disease,ethnicity_BLACK/AFRICAN AMERICAN,insurance_Government,insurance_Medicaid,Acute myocardial infarction,Pleurisy; pneumothorax; pulmonary collapse,Conduction disorders,marital_status_UNKNOWN (DEFAULT)
0,211552,0,0,0,1,1,1.0,0,0.0,0,...,0.0,0,0,0.0,0.0,0.0,1,0,0,0.0
1,294638,0,0,0,1,0,0.0,1,0.0,1,...,0.0,0,0,0.0,0.0,0.0,0,0,0,0.0
2,228232,0,0,0,0,0,0.0,0,0.0,1,...,0.0,0,0,0.0,0.0,0.0,0,0,0,0.0
3,220597,1,0,0,0,1,1.0,0,0.0,0,...,0.0,0,0,0.0,0.0,1.0,0,0,0,0.0
4,232669,1,0,0,0,0,1.0,0,0.0,0,...,0.0,0,0,0.0,0.0,0.0,0,0,0,0.0


In [35]:
# drop patient and hospital stay ID's

ptnt_demog_data2.drop(['hadm_id', 'subject_id', 'seq_num'], axis=1, inplace=True)

In [36]:
date_time_cols = ['dob', 'admittime', 'dischtime', 'intime', 'outtime', 'deathtime']
ptnt_demog_data2 = data_utils.cast_dates_and_times(ptnt_demog_data2, date_time_cols)

converting dob
converting admittime
converting dischtime
converting intime
converting outtime
converting deathtime


In [37]:
#print(ptnt_demog.columns)
print("calling calculate_durations")
ptnt_demog_data2 = ptnt_demog.calculate_age_icu_and_hospital_stay_durations(ptnt_demog_data2)
#print "ptnt_demog2 out of function"

calling calculate_durations
Calculating ages, duration of stays


In [42]:
# replace age greater than 100yrs
ptnt_demog_data2 = ptnt_demog.remove_age_greater_than_100yrs(ptnt_demog_data2)


In [44]:
print("calling create_diagnoses_defs")
diagnoses_bm_list, diagnoses = ptnt_demog.create_diagnoses_defs(ptnt_demog_data2)
#print(diagnoses_bm)
#print(diagnoses.head())

calling create_diagnoses_defs
creating diagnoses definitions
creating diagnoses definitions
map created


In [23]:
print("calling create_diagnoses_df")
diagnoses2 = ptnt_demog.create_diagnoses_df(ptnt_demog_data2, diagnoses_bm_list, diagnoses)
#print(ptnt_demog_data.head())


calling create_diagnoses_df


In [46]:
# Drop no longer useful diagnosis code columns from demographics table
diagnosis_cols = [
                    'icd9_code', 
                    'icd9_code.1', 
                    'short_title'
                    ]
# Drop diagnosis columns
ptnt_demog_data2.drop(diagnosis_cols, axis=1, inplace=True)


In [47]:
columns_by_data_type_dict = data_utils.group_column_names_by_data_type(ptnt_demog_data2)

In [48]:
columns_by_data_type_dict

{'float64_cols': ['age', 'icu_stay_duration', 'hosp_stay_duration'],
 'categorical_cols': ['first_careunit',
  'gender',
  'marital_status',
  'ethnicity',
  'insurance',
  'admission_type'],
 'bool_cols': [],
 'int64_cols': ['hospital_expire_flag'],
 'datetime64[ns]': ['admittime',
  'dischtime',
  'intime',
  'outtime',
  'deathtime',
  'dob']}

In [49]:
# Drop no longer useful date-times from demographics table

ptnt_demog_data2.drop(columns_by_data_type_dict['datetime64[ns]'],
                                                axis=1, inplace=True)

In [50]:
print("Calling continuous to categorical conversion")
ptnt_demog_data2 = data_utils.continuous_to_categorical(ptnt_demog_data2, columns_by_data_type_dict)
#print(ptnt_demog_data.head())

Calling continuous to categorical conversion
               age  icu_stay_duration  hosp_stay_duration
count  1562.000000        1562.000000         1562.000000
mean     63.442382         128.954545          315.981434
std      16.348764         180.134927          323.436033
min      18.000000          25.000000            9.000000
25%      53.000000          43.000000          121.000000
50%      67.000000          70.500000          201.000000
75%      77.000000         129.750000          381.750000
max      89.000000        1948.000000         2583.000000


### RE-ORDER COLUMNS

In [67]:
cols = list(ptnt_demog_data2.columns)
cols.insert(0, cols.pop(cols.index('hospital_expire_flag')))
ptnt_demog_data2 = ptnt_demog_data2[cols]
ptnt_demog_data2.columns

Index(['hospital_expire_flag', 'first_careunit', 'gender', 'marital_status',
       'ethnicity', 'insurance', 'admission_type', 'age', 'icu_stay_duration',
       'hosp_stay_duration'],
      dtype='object')

In [75]:
print("Calling categorical to dummy variables")
ptnt_demog_feature_cols = ptnt_demog_data2.columns[1:]
ptnt_demog_features_one_hot_df = data_utils.one_hot_encode_categorical_features(ptnt_demog_data2, ptnt_demog_feature_cols)


Calling categorical to dummy variables


In [84]:
ptnt_demog_features_one_hot_df = ptnt_demog_features_one_hot_df.merge(diagnoses2, left_index = True, right_index = True, 
                           how = 'left')

In [85]:
ptnt_demog_features_one_hot_df.head()

,hospital_expire_flag,first_careunit_CCU,first_careunit_CSRU,first_careunit_MICU,first_careunit_SICU,first_careunit_TSICU,gender_F,gender_M,marital_status_DIVORCED,marital_status_MARRIED,...,Pleurisy; pneumothorax; pulmonary collapse,Respiratory failure; insufficiency; arrest (adult),Other lower respiratory disease,Other upper respiratory disease,Other liver diseases,Gastrointestinal hemorrhage,Acute and unspecified renal failure,Chronic kidney disease,Complications of surgical procedures or medical care,Shock
icustay_id,,,,,,,,,,,,,,,,,,,,,
211552,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
294638,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228232,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
220597,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
232669,1,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [93]:
feature_selection_parameters_dict = {
                                'k': 'all',
                                'p_val': 0.1,
                                'scorer': 'chi2'
}

In [101]:
features_ranked_by_p_value = feature_selection.select_k_best_features(
                                        ptnt_demog_features_one_hot_df, 
                                        feature_selection_parameters_dict)

Selecting features with p-values < 0.1

Filtering features to include only those with p-val < 0.1.
resulting in 24 features


In [102]:
features_ranked_by_p_value

icu_stay_duration_Q3                                  4.286650e-13
Septicemia (except in labor)                          4.259116e-08
first_careunit_CSRU                                   1.048401e-06
admission_type_ELECTIVE                               2.759577e-06
hosp_stay_duration_Q3                                 8.729367e-06
age_Q3                                                2.735699e-05
icu_stay_duration_Q0                                  4.985801e-05
Respiratory failure; insufficiency; arrest (adult)    9.388291e-05
Coronary atherosclerosis and other heart disease      4.305259e-04
first_careunit_MICU                                   7.197505e-04
age_Q0                                                1.545577e-03
insurance_Private                                     2.086310e-03
insurance_Medicare                                    4.297359e-03
ethnicity_UNKNOWN/NOT SPECIFIED                       6.230196e-03
age_Q1                                                8.201892

In [ ]:
features_selected_by_p_value = feature_selection.filter_best_features_by_pvals(
                            features_ranked_by_p_value, 
                            feature_selection_parameters_dict):

In [ ]:
write_best_features(dummies)
print("Patient demographic pre-processing and feature selection complete")

In [ ]:
for col in ptnt_demog_data2.columns: 
    print(col)

In [ ]:

print("patient demographics with unique icustays")


# create patient demographic table with unique icustays as rows

dates_and_times = ['dob', 'admittime', 'dischtime', 'intime', 'outtime', 'deathtime']
for thing in dates_and_times:
    print("converting {}".format(thing))
    new_series = pd.to_datetime(ptnt_demog2.loc[:,thing])
    ptnt_demog2.loc[:,thing] = new_series
    
    '''
    rows = ptnt_demog2.index
    for row in rows:
        ptnt_demog2.loc[:,thing] = pd.to_datetime(ptnt_demog2.loc[:,thing])
    '''
print("converted to date times")
display(ptnt_demog2.head())

In [ ]:
ptnt_demog2.dtypes

In [ ]:
ptnt_demog2.shape

In [ ]:
print "calculating ages, duration of stays"
# len(pd.date_range()) APPEARS TO TAKE A VERY LONG TIME
for index, row in ptnt_demog2.iterrows():
    if (pd.notnull(row['intime']) & pd.notnull(row['dob'])):
        #age_val = relativedelta(row['intime'], row['dob']).years
        # calculating year end frequency
        age_val = len(pd.date_range(end = row['intime'], start = row['dob'], freq = 'A'))
    else: 
        age_val = np.nan
    if (pd.notnull(row['intime']) & pd.notnull(row['outtime'])):
        #delta_time = relativedelta(row['outtime'], row['intime'])
        #icu_stay_val = delta_time.days*24 + delta_time.hours
        icu_stay_val = len(pd.date_range(end = row['outtime'], start = row['intime'], freq = 'H'))
    else: 
        icu_stay_val = np.nan
    if (pd.notnull(row['admittime']) & pd.notnull(row['dischtime'])):
        #delta_time = relativedelta(row['dischtime'], row['admittime'])
        #hosp_stay_val = delta_time.days*24 + delta_time.hours
        hosp_stay_val = len(pd.date_range(end = row['dischtime'], start = row['admittime'], freq = 'H'))
    else:
        hosp_stay_val = np.nan
    
    ptnt_demog2.set_value(index, 'age', age_val)
    ptnt_demog2.set_value(index, 'icu_stay', icu_stay_val)
    ptnt_demog2.set_value(index, 'hosp_stay', hosp_stay_val)

cols = list(ptnt_demog2.columns)
cols.pop(cols.index('icd9_code'))
cols.pop(cols.index('icd9_code.1'))
cols.pop(cols.index('short_title'))
cols.pop(cols.index('intime'))
cols.pop(cols.index('outtime'))
cols.pop(cols.index('admittime'))
cols.pop(cols.index('dischtime'))
cols.pop(cols.index('seq_num'))
cols.pop(cols.index('dob'))

#cols.insert(0, cols.pop(cols.index('icustay_id')))
cols.insert(0, cols.pop(cols.index('hadm_id')))
cols.insert(1, cols.pop(cols.index('age')))
cols.insert(2, cols.pop(cols.index('icu_stay')))
cols.insert(3, cols.pop(cols.index('hosp_stay')))
cols.insert(len(cols), cols.pop(cols.index('hospital_expire_flag')))
display(cols)


ptnt_demog2 = ptnt_demog2[cols]
print "patient demographics with calculated ages, duration of stays"
display(ptnt_demog2.head())

## DROP OUTLIER AGE VALUES

In [ ]:
ptnt_demog2['age'].describe()

In [ ]:
age_replace_vals = list(ptnt_demog2[ptnt_demog2['age'] > 110]['age'].unique())
display(age_replace_vals)

ptnt_demog2['age'].replace(age_replace_vals, np.nan, inplace = True)
ptnt_demog2['age'].dropna().describe()

## DISPLAY DESCRIPTIVE STATS

In [ ]:
ptnt_demog2.columns

In [ ]:
display(ptnt_demog2.icu_stay.dropna().describe())
display(ptnt_demog2.hosp_stay.dropna().describe())
display(ptnt_demog2[ptnt_demog2.icu_stay <= 0])
display(ptnt_demog2[ptnt_demog2.hosp_stay <=0])
icu_stay_low = list(ptnt_demog2[ptnt_demog2['icu_stay'] <=0]['icu_stay'].unique())
age_replace_vals = list(ptnt_demog2[ptnt_demog2['age'] > 110]['age'].unique())

#ptnt_demog2.loc[:,'subject_id'] = ptnt_demog2.index
#ptnt_demog2.index = ptnt_demog2.icustay_id
#ptnt_demog2.drop('icustay_id', axis = 1, inplace = True)
#ptnt_demog2.head()

Date and time data imported in string format is converted to pandas.datetime objects

## Extracting diagnoses information using code grabbed from benchmarking exercise

In [ ]:
#phenotypes = add_hcup_ccs_2015_groups(diagnoses, yaml.load(open(args.phenotype_definitions, 'r')))
definitions = yaml.load(open('hcup_ccs_2015_definitions.yaml', 'r'))

diagnoses = ptnt_demog[['hadm_id', 'icd9_code', 'short_title']].copy()

# create mapping of hcup_ccs_2015_definitions to diagnoses icd9 codes
def_map = {}
for dx in definitions:
    for code in definitions[dx]['codes']:
        def_map[code] = (dx, definitions[dx]['use_in_benchmark'])

print "map created"
# map hcup_ccs_2015 definitions to icd9 diagnoses codes
diagnoses['HCUP_CCS_2015'] = diagnoses.icd9_code.apply(lambda c: def_map[c][0] if c in def_map else None)
diagnoses['USE_IN_BENCHMARK'] = diagnoses.icd9_code.apply(lambda c: int(def_map[c][1]) if c in def_map else None)
#diagnoses['subject_id'] = diagnoses.index
#diagnoses.set_index(np.arange(diagnoses.shape[0]), inplace = True)


# create dataframe from the def_map dict so that we can isolate the 
# definitions that are used in benchmarking

def_map_df = pd.DataFrame.from_dict(def_map, orient = 'index')
def_map_df.columns = ['Diagnoses', 'Benchmark']
diagnoses_bm = list(def_map_df[def_map_df.Benchmark == True].drop_duplicates('Diagnoses').Diagnoses)
icustays = list(ptnt_demog2.index)

# create dataframe with hcup_ccp diagnoses benchmark categories as columns and
# icustay_id information as indices. if the diagnosis is present for a given icustay the 
# value is 1, otherwise 0. 

diagnoses2 = pd.DataFrame(columns = diagnoses_bm, index = icustays)
diagnoses2.fillna(0, inplace = True)
print "created empty diagnosis dataframe"
for row in diagnoses.iterrows():
    if row[1]['USE_IN_BENCHMARK'] == 1:
        diagnoses2.loc[row[0]][row[1]['HCUP_CCS_2015']] = 1

print "filled diagnosis dataframe"
diagnoses2.head()

In [ ]:
ptnt_demog2.drop(['subject_id', 'deathtime', 'hadm_id'], inplace = True, axis = 1)
cols = list(ptnt_demog2.columns)
cols.insert(0, cols.pop(cols.index('hospital_expire_flag')))
ptnt_demog2 = ptnt_demog2[cols]


In [ ]:
diagnoses.dtypes

In [ ]:
diagnoses2.columns


## CONVERT CONTINUOUS TO CATEGORICAL DATA

In [ ]:
# NEED TO DO SOME OVERVIEW OF DATA INCLUDING DISTRIBUTIONS, SUMMARY STATISTICS ETC
# SINCE WE HAVE SO MUCH DATA, WE ARE JUMPING AHEAD..... IN TIME!! 
# AND SELECTING FEATURES THAT WERE USED IN THE FINAL CLASSIFICATION

demog_select_features_df = ptnt_demog2[['hospital_expire_flag', 'icu_stay', 'age', 'first_careunit', 'admission_type']]
demog_select_features_df.head()

demog_select_features_df[demog_select_features_df.columns[:3]].dropna().groupby('hospital_expire_flag').describe()


pd.DataFrame(demog_select_features_df.groupby('hospital_expire_flag').first_careunit.value_counts()).plot.bar(stacked = True)

icu_admits_df = pd.DataFrame(demog_select_features_df['first_careunit'].value_counts()/demog_select_features_df.shape[0])
icu_admits_df.index

icu_mortality_rates_df = pd.DataFrame(demog_select_features_df.groupby('first_careunit').hospital_expire_flag.sum()/ \
demog_select_features_df.groupby('first_careunit').hospital_expire_flag.count())
icu_admits_df.merge(icu_mortality_rates_df, left_index = True, right_index = True, 
                       how = 'left', sort = True)

demog_select_features_df[demog_select_features_df['first_careunit'] == 'MICU']['first_careunit'].count()



admission_types_df = pd.DataFrame(demog_select_features_df['admission_type'].value_counts()/demog_select_features_df.shape[0])

admission_type_mortality_rates_df = pd.DataFrame(demog_select_features_df.groupby('admission_type').hospital_expire_flag.sum()/ \
demog_select_features_df.groupby('admission_type').hospital_expire_flag.count())

admission_types_df.merge(admission_type_mortality_rates_df, left_index = True, right_index = True, 
                       how = 'left', sort = True)
'''
earlier versions included these factors but the  process has been iterative so we 
pre-selected relevant features above
diagnoses2['GCS Total_15', 'GCS Total_3', 'GCS Total_6', 'GCS Total_4', 
                          'Respiratory Failure','Shock', 'Septicemia (except in labor)', 'Acute and unspecified renal failure','Other liver diseases',
                         'Fluid and electrolyte disorders',
                          'Pneumonia (except that caused by tuberculosis or sexually transmitted disease)',
                            'Acute cerebrovascular disease']
'''

In [ ]:
def quant_cats(feature, Q1, Q2, Q3):
    if feature <=Q1:
        return 'Q0'
    elif (feature >Q1 and feature <= Q2):
        return 'Q1'
    elif (feature > Q2 and feature <= Q3):
        return 'Q2'
    elif feature > Q3:
        return 'Q3'
    

ptnt_demog3 = ptnt_demog2.copy()
demog_stats = ptnt_demog3[ptnt_demog3.columns[1:4]].dropna().describe()
for col in ptnt_demog3.columns[1:4]:
        Q1 = demog_stats[col].loc['25%']
        Q2 = demog_stats[col].loc['50%']
        Q3 = demog_stats[col].loc['75%']
        ptnt_demog3[col] = ptnt_demog3[col].apply(lambda x: quant_cats(x, Q1, Q2, Q3))



In [ ]:
demog_stats

## CONVERT CATEGORICAL TO DUMMY VARIABLES

In [ ]:
ptnt_demog2.dtypes


In [ ]:
ptnt_demog3.dtypes


In [ ]:
dummies = ptnt_demog2[ptnt_demog3.columns[:1]]

#display(monkey.head())
for col in ptnt_demog3.columns[1:]:
    chimp = pd.get_dummies(ptnt_demog3[col], prefix = col)
    dummies = dummies.merge(chimp, left_index = True, right_index = True, 
                       how = 'left', sort = True)

dummies.head()
    

## MERGE DUMMY VARIABLES AND DIAGNOSES

dummies = dummies.merge(diagnoses2, left_index = True, right_index = True, 
                       how = 'left')
dummies.dropna().shape

In [ ]:
diags2 = list(diagnoses2.columns)
#diags.pop(diags.index('Diabetes mellitus without complication'))
diags = [
 'hospital_expire_flag',
 'Acute cerebrovascular disease',
 'Other liver diseases',
 'Acute and unspecified renal failure',
 'Fluid and electrolyte disorders',
 'Septicemia (except in labor)',
 'Pneumonia (except that caused by tuberculosis or sexually transmitted disease)',
 'Respiratory failure; insufficiency; arrest (adult)',
 'Shock'
 ]


plt.subplots(figsize=(13,6))
diags_df = dummies[diags]
#diags_df.head()

diags_df[diags_df.columns[1:]][diags_df['hospital_expire_flag']==1].sum().plot.bar(
        alpha=0.5,label= 'Non_Survival')
diags_df[diags_df.columns[1:]][diags_df['hospital_expire_flag']==0].sum().plot.bar(
        alpha=0.1,label= 'Survival')
    

## SELECT FEATURES AND WRITE TO FILE. 

In [ ]:


root = os.getcwd() + '/features/'
name = 'Ptnt_Demog_Features.csv'
name2 = 'Ptnt_Demog_FeaturesScores.csv'
frame = dummies
X = frame[frame.columns[1:]]
y = frame['hospital_expire_flag']

        
# SELECT K BEST FEATURES BASED ON CHI2 SCORES
selector = SelectKBest(score_func = chi2, k = 'all')
selector.fit(X, y)
p_vals = pd.Series(selector.pvalues_, name = 'p_values', index = X.columns)
scores = pd.Series(selector.scores_, name = 'scores', index = X.columns)
features_df = pd.concat([p_vals, scores], axis = 1)
features_df.sort_values(by ='scores', ascending = False, inplace = True)
print "Feature scores/p_values in descending/ascending order"
display(features_df.head(20))

best_features = frame[features_df[features_df.p_values < .001].index]

frame = pd.DataFrame(y).merge(best_features, left_index = True, right_index = True, 
                how = 'left', sort = True)


print "{}     {}".format(name, frame.shape)


# CODE TO REORDER FEATURES ALPHABETICALLY. MAKES FOR CLEANER COLUMNS, GROUPED BY TYPE
# BUT WHAT WE MAY WANT IS THE ORDER OF FEATURE SIGNIFICANCE PRESERVCED. 
'''
cols = list(frame.columns)
cols.sort()
cols.insert(0, cols.pop(cols.index('hospital_expire_flag')))
frame = frame[cols]
'''

print "head of selected feature frame "
display(frame.head())
frame.to_csv(root + name)
features_df[features_df.p_values < .001].to_csv(root + name2)
y = pd.DataFrame(y)
y.to_csv(root + 'outcomes.csv')